# Trabalho Prático 1 de Estruturas Criptográficas

- **Autores:** (Grupo 9)
     - Nelson Faria (A84727)
     - Miguel Oliveira (A83819)

## Exercício 1

In [6]:
'''
Imports necessários para a execução deste mesmo Notebook
'''
import os, io
import multiprocessing as mp
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dsa

O principal objetivo deste primeiro exercício prático passava pela implementação de uma **comunicação síncrona** e **segura** entre duas entidades, um **receiver** e um **emitter** seguindo determinados parâmetros que vão sendo explicitados e demonstrados em cada uma das alíneas que se seguem.

### Alínea a - Implementar um gerador de nounces

Como se encontra explícito no próprio título desta subsecção, a principal finalidade desta alínea era a implementação de um **gerador de nounces** de modo a que, sempre que seja solicitado uma nova instância de **nounce**, temos a garantia de que ele é **aleatório** e ainda que **nunca foi retornado**, numa ocasião prévia por esse mesmo gerador.

Com isto em mente, desenvolvemos a seguinte classe em **Python**, designada de **NGenerator**, a qual apenas apresenta um método que permite o acesso a um novo **nounce**, designado de **get()**. De modo a implementar essa funcionalidade no **NGenerator**, recorremos a uma lista que vai armazenando os **nounces** que vão sendo retornados, à qual acedemos sempre aquando da solicitação da geração de um novo nounce de modo a verificar se tal nounce já foi retornado previamente. Em caso afirmativo, essa instância recém-gerada **é descartada** e é gerada **uma nova instância** para ser retornada, adicionando-a, previamente ao *historic* do respetivo **NGenerator**.

In [5]:
''' 
Classe que permite gerar 
nounces que nunca se repetem
'''
class NGenerator :

    '''
    Construtor para objetos da classe NGenerator
    '''
    def __init__(self) :
        self.historic = []

    ''' 
    Método que nos permite obter um novo nounce
    '''
    def get(self) :
        nounce = os.urandom(12)
        while nounce in self.historic :
            nounce = os.urandom(12)
        self.historic.append(nounce)
        return nounce

> De seguida, mostramos o exemplo de execução do método que permite obter uma nova instância de **nounce**.

In [9]:
ng = NGenerator()
ng.get()

b'\x04\xbcg\xf9p\xba\x9d\x18\xb0\xcc\x0e\xd6'

### Exemplo simples daquilo que será a comunicação: (Ainda sem qualquer requisito de segurança pedido)

[FIXME]

In [7]:
''' 
Função que representa a execução da entidade sender
'''
def sender(conn,stdin):
    message = stdin.recv()

    while len(message) > 0 :
        conn.send(message.encode('utf-8'))
        assert 'ok' == conn.recv()     # para garantir o sincronismo
        ''' Pedimos à thread principal a 
        próxima mensagem lida a partir do stdin'''
        stdin.send('next')
        message = stdin.recv()

    # Para terminar a conexão
    conn.send(message.encode('utf-8'))
    conn.close()
    print('[SENDER] SHUTDOWN')

''' 
Função que representa a execução da entidade receiver 
'''
def receiver(conn):
    try:
        message = bytes(conn.recv()).decode('utf-8')
    except EOFError:
        print('EOFError')
    while len(message) > 0:
        # Imprimimos o conteudo da mensagem recebida
        print('[Receiver] RECEIVED: ' + message)
        conn.send('ok')        # para garantir o sincronismo
        try:
            message = bytes(conn.recv()).decode('utf-8')
        except EOFError:
            break
    conn.close()
    print('[RECEIVER] SHUTDOWN')


''' 
Função responsável por arrancar a 
execução de ambas as entidades, o emitter 
e o receiver. Para além disto é responsável por 
mediar a comunicação entre o stdin e o sender
'''
def main() :
    try:
        mp.set_start_method('fork')
    except:
        print("O start_method já foi inicializado anteriormente ")

    receiver_conn, sender_conn = mp.Pipe()
    receiver_stdin, main_stdin = mp.Pipe()
    s = mp.Process(target=sender, args=(sender_conn,receiver_stdin))
    r = mp.Process(target=receiver, args=(receiver_conn,))
    s.start()
    r.start()

    message = input('[Emitter] > Mensagem a ser enviada: \n')
    while len(message) != 0 and message != 'exit':
        main_stdin.send(message)
        ''' Esperamos a confirmação do sender para 
        receber nova mensagem do stdin'''
        assert 'next' == main_stdin.recv()
        message = input('[Emitter] > Mensagem a ser enviada: \n')
    # Para terminar a ligação
    main_stdin.send('')

In [8]:
main()

O start_method já foi inicializado anteriormente 


[Emitter] > Mensagem a ser enviada: 
 olá tudo bem oh receiver?


[Receiver] RECEIVED: olá tudo bem oh receiver?


[Emitter] > Mensagem a ser enviada: 
 Não falas não é?


[Receiver] RECEIVED: Não falas não é?


[Emitter] > Mensagem a ser enviada: 
 Vou ali e já venho...


[Receiver] RECEIVED: Vou ali e já venho...


[Emitter] > Mensagem a ser enviada: 
 exit


[RECEIVER] SHUTDOWN[SENDER] SHUTDOWN

